In [1]:
from IPython.display import Latex

_This notebook goes along with the C++ files in the github repository xababax.
Together with the code, they present a solution to the Riddler puzzle:
https://fivethirtyeight.com/features/can-you-unravel-these-number-strings/
I used this riddle to teach myself some C++ as well as Gnuplot, your comments and suggestions are appreciated. My email address is in the cpp file._

# The riddle: unravelling the number string

The details of the riddle are available on the link to the riddle above. The definition of the string is the following:

The string is an infinite string of 3's and 2's and is ordered in such a way that the i-th digit indicates the number of consecutive 3's that are placed immediately before the i-th '2'.

The challenge here is to calculate the ratio of 3's to 2's in the sequence.

## A few definitions

As a starting point, let's start with some notations:

$(S_i)_{i>0}$ is the sequence $333233323332332...$

$ R_2 $ is the ratio of 2's in the entire sequence.

$ R_3 = 1-R_2 $ is therefore the ratio of 3's.


Later on, we'll extend the definition of $(S_i)$ to sequences containing an arbitrary base number $n>2$ and a delimitor $m$.


## A few observations


There is only one sequence that satisfies the definition and it can be constructed starting from the first digit (which has to be 3) with the following procedure:

In [3]:
seq_three = '3332'
seq_two = '332'

S = '3'
i=0
while i<10:
    if S[i] == '3': S+=seq_three
    else: S+=seq_two
    i+=1
print(S)

333323332333233323323332333233323323332


Since $ (S_i) $ can only contain 3's and 2's, it can only be composed of substrings of the form $(3332)$ and $(332)$. As a result, we can immediately find the following bounds:

$ \frac{1}{4} < R_2 < \frac{1}{3}  $



Finally, one can see that the sequence cannot be periodic by the following argument,
suppose $S_i$ is periodic:

$S_i$ is periodic $\implies \exists! N > 0$ such that:
- $\forall k>0:$  $S_k = S_{k+N}$.
- $N$ is the smallest integer with this property.

$\forall p>0$, $S_p, S_{p+1}, ... , S_{p+4N}$, by virtue of the building algorithm above $\exists q>0$ s.t. a subset of $S_q, S_{q+1}, ... , S_{q+F}$  where ($F<N$) generated $S_p, S_{p+1}, ... , S_{p+4N}$.

$S_k = S_{k+N} \implies \exists T<F<N$ such that within $S_q, S_{q+1}, ... , S_{q+F}$, we have $S_k = S_{k+T}$.

Since p was arbitrary, this implies that T is another period for $(S_i)_{i>0}$. QED

## An exact solution

Using the building algorithm, one can also calculate exactly $R_2$.

Each '3' generates three 3's and one 2 while each '2' generates two 3's and one 2.

In matrix terms, the generator of the number of 3's and 2's generated after n iterations can be written as:
$M = \begin{pmatrix}
3 & 1\\
2 & 1
\end{pmatrix}$

With this notation, we have:
$M^n * \begin{pmatrix}
1 \\
0
\end{pmatrix}$
=
$\begin{pmatrix}
3_n \\
2_n
\end{pmatrix}$

In the basis where $\begin{pmatrix}
1 \\
0
\end{pmatrix}$ corresponds to the number of 3's and $\begin{pmatrix}
0 \\
1
\end{pmatrix}$ corresponds to the number of 2's.

The matrix can be diagnoalized and $2_n$ and $3_n$ expressed exactly. With this notation, we find:

$R_2 = \displaystyle\lim_{n \to \inf} \frac{2_n}{2_n + 3_n} = 2 - \sqrt 3$

And therefore in the terms of the riddler's challenge the ratio of 3's to 2's is:

$ R_{3/2} = 1 + \sqrt 3 $

# A numerical approach in C++

I used this opportunity to teach myself a little bit of C++ and gnuplot.

num-string.cpp (found in the repository) computes the string for different base numbers n (equivalent to 3 in the riddler's challenge) and delimitors (equivalent to 2 in the riddler challenge).

For the problem to be defined, it is necessary for both base and delimitor to non null, and to be different to each other. num-string.cpp takes that into account.

After compiling, it produces as output the a space delimited file wich outputs are $n$ (base), $m$ (delimitor), $r$ (which in the previous section's notation is $ R_m $), and $c= R_{n/m}$.



I used the output of the program to learn a bit about gnuplot. I was more interested in $R_m$ than R_{n/m}, so the plots below reflect this. They also include some interpolations.

!["a"](ratio_vs_base_digits.png)

In this plot, for each base digit $n \in [2:9]$, I plot $R_m$ for $m \in [2:n]$ and included an interpolation of the average ratio by base digit $n$.
The Gnuplot code is below (clearly there is a lot of room for improvement):

set title "Ratio of delimitor digits: base digit from 1 to 9, delimitor digit from 1 to base"

R(x) = 1/x - 1/(2x*x)

set xlabel "Base digit"

set ylabel "Ratio"

set key title "R(x) = 1/x - 1/(2*x*x)"

plot "results.log" using 1:3 title "Constructed string", [x=2:9] 1/x-1/(2*x*x) title "Interpolated"

 !["b"](Rm_vs_n_m.png)

In this plot, for each base digit $n \in [2:9]$, I plot $R_m$ for $m \in [2:9]$, and $n \neq m$ in a 3D plot using the code below in Gnuplot.

set key font "Arial, 12"

set key spacing 1.8

set ylabel "Delimitor digit"

set xlabel "Base digit"

set zlabel "Ratio"

set ylabel font "Arial, 13"

set xlabel font "Arial, 13"

set zlabel font "Arial, 13"

set title "Unravelling the number string"

set title font "Arial, 14"

splot [x=1:20] [y=1:20] 1.0/(x+sqrt(y)) with dots, "results.log" using 1:2:3 title "computed string"